In [118]:
import sys
sys.path.append("..")

# Reload modules in /src/ when changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
df = pd.read_pickle('../data/resampled/EURUSD-5m-20240101-20241231.pkl')

In [120]:
df.head()

,timestamp,open,high,low,close,volume,spread
0,2024-01-01 22:00:00,1.104540,1.104555,1.104505,1.104525,1.414000e+11,0.000510
1,2024-01-01 22:05:00,1.104515,1.104595,1.104515,1.104565,1.971000e+11,0.000595
2,2024-01-01 22:10:00,1.104555,1.104595,1.104545,1.104585,1.624500e+11,0.000573
3,2024-01-01 22:15:00,1.104580,1.104595,1.104545,1.104545,3.600000e+10,0.000577
4,2024-01-01 22:20:00,1.104585,1.104585,1.104440,1.104505,1.530000e+11,0.000441


In [121]:
feature_name = 'close'

In [122]:
seq_length = 30

In [123]:
feature_values = df[column_name].values
feature_values

array([1.104525, 1.104565, 1.104585, ..., 1.040445, 1.040475, 1.04058 ],
      shape=(74708,))

In [124]:
len_df = df.shape[0]
len_df

74708

## Brute Force

In [125]:
%%time
df_sequential = pd.DataFrame(columns=[f'X{i}' for i in range(seq_length)] + ['y_regr'])

for i in range(0, len_df - seq_length):
    idx_start, idx_end = i, i + seq_length + 1
    df_sequential.loc[len(df_sequential)] = feature_values[idx_start:idx_end]

CPU times: user 42.5 s, sys: 17.8 s, total: 1min
Wall time: 1min


## Vectorized

In [134]:
%%time
list_slices = [feature_values[i:len_df-seq_length+i] for i in range(0, seq_length + 1)]
combined_slices = np.vstack(list_slices).T

df_sequential = pd.DataFrame(combined_slices, columns=[f'{feature_name}_{i}' for i in range(seq_length)] + ['y_regr'])
df_sequential

CPU times: user 1.83 ms, sys: 2.91 ms, total: 4.74 ms
Wall time: 3.62 ms


,close_0,close_1,close_2,close_3,close_4,close_5,close_6,close_7,close_8,close_9,...,close_21,close_22,close_23,close_24,close_25,close_26,close_27,close_28,close_29,y_regr
0,1.104525,1.104565,1.104585,1.104545,1.104505,1.104500,1.104500,1.104400,1.104415,1.104400,...,1.10359,1.103680,1.103670,1.103770,1.103790,1.103725,1.103700,1.103480,1.103485,1.103575
1,1.104565,1.104585,1.104545,1.104505,1.104500,1.104500,1.104400,1.104415,1.104400,1.104490,...,1.10368,1.103670,1.103770,1.103790,1.103725,1.103700,1.103480,1.103485,1.103575,1.103565
2,1.104585,1.104545,1.104505,1.104500,1.104500,1.104400,1.104415,1.104400,1.104490,1.104455,...,1.10367,1.103770,1.103790,1.103725,1.103700,1.103480,1.103485,1.103575,1.103565,1.103585
3,1.104545,1.104505,1.104500,1.104500,1.104400,1.104415,1.104400,1.104490,1.104455,1.104085,...,1.10377,1.103790,1.103725,1.103700,1.103480,1.103485,1.103575,1.103565,1.103585,1.103785
4,1.104505,1.104500,1.104500,1.104400,1.104415,1.104400,1.104490,1.104455,1.104085,1.104045,...,1.10379,1.103725,1.103700,1.103480,1.103485,1.103575,1.103565,1.103585,1.103785,1.103820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74673,1.040020,1.040060,1.040005,1.040130,1.039990,1.040065,1.040170,1.040395,1.040360,1.040485,...,1.04056,1.040540,1.040160,1.039920,1.040130,1.040225,1.040280,1.040325,1.040505,1.040555
74674,1.040060,1.040005,1.040130,1.039990,1.040065,1.040170,1.040395,1.040360,1.040485,1.040720,...,1.04054,1.040160,1.039920,1.040130,1.040225,1.040280,1.040325,1.040505,1.040555,1.040510
74675,1.040005,1.040130,1.039990,1.040065,1.040170,1.040395,1.040360,1.040485,1.040720,1.040740,...,1.04016,1.039920,1.040130,1.040225,1.040280,1.040325,1.040505,1.040555,1.040510,1.040445
74676,1.040130,1.039990,1.040065,1.040170,1.040395,1.040360,1.040485,1.040720,1.040740,1.040310,...,1.03992,1.040130,1.040225,1.040280,1.040325,1.040505,1.040555,1.040510,1.040445,1.040475


## Multivariate

In [100]:
%%time
seq_len = 10
feature_columns = ['open', 'high', 'low', 'close', 'volume', 'spread']

columns = np.vstack(list(map(lambda feature_name: [f'{feature_name}_{i}' for i in range(seq_length)], feature_columns))).reshape(-1)

mv_slice_values = np.vstack([[df[feature_name].values[i:len_df-seq_length+i] for i in range(0, seq_length)] for feature_name in feature_columns]).T

df_sequential_mv = pd.DataFrame(
    mv_slice_values,
    columns=columns
)

df_sequential_mv

CPU times: user 11 ms, sys: 13.6 ms, total: 24.6 ms
Wall time: 23 ms


,open_0,open_1,open_2,open_3,open_4,open_5,open_6,open_7,open_8,open_9,...,spread_20,spread_21,spread_22,spread_23,spread_24,spread_25,spread_26,spread_27,spread_28,spread_29
0,1.104540,1.104515,1.104555,1.104580,1.104585,1.104510,1.104505,1.104475,1.104390,1.104390,...,0.000029,0.000034,0.000029,0.000030,0.000035,0.000029,0.000021,0.000032,0.000025,0.000030
1,1.104515,1.104555,1.104580,1.104585,1.104510,1.104505,1.104475,1.104390,1.104390,1.104395,...,0.000034,0.000029,0.000030,0.000035,0.000029,0.000021,0.000032,0.000025,0.000030,0.000029
2,1.104555,1.104580,1.104585,1.104510,1.104505,1.104475,1.104390,1.104390,1.104395,1.104480,...,0.000029,0.000030,0.000035,0.000029,0.000021,0.000032,0.000025,0.000030,0.000029,0.000028
3,1.104580,1.104585,1.104510,1.104505,1.104475,1.104390,1.104390,1.104395,1.104480,1.104495,...,0.000030,0.000035,0.000029,0.000021,0.000032,0.000025,0.000030,0.000029,0.000028,0.000026
4,1.104585,1.104510,1.104505,1.104475,1.104390,1.104390,1.104395,1.104480,1.104495,1.104090,...,0.000035,0.000029,0.000021,0.000032,0.000025,0.000030,0.000029,0.000028,0.000026,0.000027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74673,1.040170,1.040030,1.040065,1.040010,1.040135,1.039995,1.040075,1.040175,1.040385,1.040365,...,0.000230,0.000213,0.000219,0.000169,0.000082,0.000080,0.000048,0.000048,0.000043,0.000031
74674,1.040030,1.040065,1.040010,1.040135,1.039995,1.040075,1.040175,1.040385,1.040365,1.040480,...,0.000213,0.000219,0.000169,0.000082,0.000080,0.000048,0.000048,0.000043,0.000031,0.000030
74675,1.040065,1.040010,1.040135,1.039995,1.040075,1.040175,1.040385,1.040365,1.040480,1.040755,...,0.000219,0.000169,0.000082,0.000080,0.000048,0.000048,0.000043,0.000031,0.000030,0.000035
74676,1.040010,1.040135,1.039995,1.040075,1.040175,1.040385,1.040365,1.040480,1.040755,1.040440,...,0.000169,0.000082,0.000080,0.000048,0.000048,0.000043,0.000031,0.000030,0.000035,0.000033
